# Dependency Trees

THIS IS A SKETCH, FOR DISCUSSION!

## Introduction

Let's make an API for dependency trees.

Dependency grammars describe relationships between lexical units in a sentence.  *Unit* here means signifier: sound/meaning.  For many languages these are *words*, which I take to be mostly a phonological concept.  For agglutinative languages such as those in the Altaic and Uralic families (Turkish, Finnish), the phonological word is too large a unit: instead, we would want to work with *morphemes*.  So instead we simply use the term *form*.  Sign = Form + meaning.

Dependencies are binary asymmetric relations between pairs of forms: a *governor* or *head* and a *dependent*.  What exactly are these relations?  There are many varieties of dependency grammar, taking different sets of relations as primaries of the grammar.  Nivre summarizes the possibilities, for a governor H and a depedent D:

1. H determines the syntactic category of  C and can often replace C. 
2. H determines the semantic category of C; D gives semantic specification. 
3. H is obligatory; D may be optional. 
4. H selects D and determines whether D is obligatory or optional. 
5. The form ofD depends on H (agreement or government).
6. The linear position of D is specified with reference to H.

The simplest approach is to simply not state the dependency type overtly, *untyped dependencies*.  In this case the analysis  just yields a graph of binary relations between forms.

Typed dependencies are triples `(head, dependent, dep-relation)`.  For any sentence, each dependent has only one governor.  A dependency graph is therefore, formally, a *tree*: a connected acyclic rooted graph.

The specific sets of relations one selects in effect defines a theory of grammar.  There is much confusion around this.  For example, Stanford's CoreNLP refers to dependency graphs as "semantic trees", but it is immediately obvious from the relations used (see below) that these are syntactic relations. If semantics were involved, the dependency trees for `All cats are grey` and `Some cats are grey` would not be identical (they are). 

The [Universal Dependencies](https://universaldependencies.org/introduction.html) developed from an older set of relations developed at Stanford, provide a core set of [cross-linguistic relations](https://universaldependencies.org/u/dep/index.html) used in annotating a large number of languages.  These are the relations (and POS tags) output by the [new neural models](https://stanfordnlp.github.io/stanfordnlp/) recently released by the Stanford NLP group.

Dependency linguistics has not, traditionally, been popular in the US, presumably because mainstream linguists find that dependency grammars fail to provide (even) descriptive accounts of syntactic phenomena like agreement and movement.  Computational linguists like dependencies because automatic parsers are easier to use, so they are willing to accept the limitations in exchange.  See [here](https://linguistics.stackexchange.com/questions/7280/why-is-constituency-needed-since-dependency-gets-the-job-done-more-easily-and-e) for a readable discussion. 

A question, then, is whether philologists might find some use for dependency parses.  To find out, we ought to provide a useful representation for working with trees.



## Desiderata

At work and in school I have found two main uses for automatic parsers:

1.  To find specific structures in some corpus of language.  Say I'd like to find occurences of passivised subjects with past tense verbs, perhaps as a shallow feature for some statistical model.  Therefore a method for searching through parses (tree searching) is required.
2.  Determining the syntactic parse of a whole sentence, with some sort of visualization.  This is more useful when e.g. studying the grammar of a language.

I'm sure there are more.  What are they?

We need a data structure to represent trees.  In its current state, the Python `stanfordnlp` project does not supply one, in contrast to the very elaborate Java [SemanticGraph](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/semgraph/SemanticGraph.html) published as part of the CoreNLP project.  

## A pythonic approach?

Python provides a tree class in its core distribution: [ElementTree](https://docs.python.org/2/library/xml.etree.elementtree.html)  Of course the module is generally used for XML processing, but we may certainly commandeer it for our purposes.  It provides everything we need:
- A data structure for nodes and their children.
- Nodes have attribute dictionaries, for representing morphosyntactic and other features.
- An [XPath](https://www.w3schools.com/xml/xml_xpath.asp) mechanism for searching trees.

Let's see how far it gets us.

Let's begin by pulling a parse from the stanford toolset.

In [192]:
import stanfordnlp

In [193]:
nlp = stanfordnlp.Pipeline()

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/jds/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/jds/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/home/jds/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/jds/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/jds/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/home/jds/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 

In [194]:
doc = nlp('In the summer of the Roman year 699, now described as the year 55 before the birth of Christ, the Proconsul of Gaul, Gaius Julius Caesar, turned his gaze upon Britain.')

The `Document` object provides access to `Sentence` objects, and these to `Word`s and dependencies:

In [195]:
type(doc), type(doc.sentences[0]), type(doc.sentences[0].words[0])

(stanfordnlp.pipeline.doc.Document,
 stanfordnlp.pipeline.doc.Sentence,
 stanfordnlp.pipeline.doc.Word)

In [196]:
doc.sentences[0].print_dependencies()

('In', '3', 'case')
('the', '3', 'det')
('summer', '31', 'obl')
('of', '7', 'case')
('the', '7', 'det')
('Roman', '7', 'amod')
('year', '3', 'nmod')
('699', '7', 'nummod')
(',', '31', 'punct')
('now', '11', 'advmod')
('described', '31', 'advcl')
('as', '14', 'case')
('the', '14', 'det')
('year', '11', 'obl')
('55', '14', 'nummod')
('before', '18', 'case')
('the', '18', 'det')
('birth', '11', 'obl')
('of', '20', 'case')
('Christ', '18', 'nmod')
(',', '31', 'punct')
('the', '23', 'det')
('Proconsul', '31', 'nsubj')
('of', '25', 'case')
('Gaul', '23', 'nmod')
(',', '27', 'punct')
('Gaius', '25', 'conj')
('Julius', '27', 'flat')
('Caesar', '27', 'flat')
(',', '31', 'punct')
('turned', '0', 'root')
('his', '33', 'nmod:poss')
('gaze', '31', 'obj')
('upon', '35', 'case')
('Britain', '31', 'obl')
('.', '31', 'punct')


The numbers in each dep are indices into the word list, determining the governor in each dependency.  So "In" is a `case` dependent of "summer'.  (It is quite correct to take prepositions as exponents of case relations, but this examples shows that universal dependencies are syntactic in nature).

`Word` objects are richly annotated with morphosyntactic information:

In [197]:
doc.sentences[0].words[10]

<Word index=11;text=described;lemma=describe;upos=VERB;xpos=VBN;feats=Tense=Past|VerbForm=Part;governor=31;dependency_relation=advcl>

For this information to be visible to XPath searches it needs to be transferred to the attributes of ElementTree nodes.  A `Form` class might accomplish this.

## Forms

A form is fundamentally just a string.  But in a grammatical analysis we'll also want morphological information tied to the form: POS and other relevant morphosyntactic features.  So instead we'll derive from Python's `Element` class, so that we can arrange forms in `ElementTree` structures.

In [198]:
from xml.etree.ElementTree import Element, ElementTree
from xml.etree.ElementTree import dump
from typing import List, Union


class Form(Element):
    def __init__(self, form : str, id :int = 0) -> None:
        Element.__init__(self, form, attrib={'id' : str(id)})
        
    def __truediv__(self, pos_tag : str) -> 'Form':
        '''
        Sets the POS feature of this form.
        '''
        self.set('pos', pos_tag)
        return self
    
    def __rshift__(self, other : Union['Form', str]) -> 'Dependency':
        '''
        Create a dependency between this form as governor, to the other as dependent.
        Adds the dependent to the children of this form.
        '''
        other = Form(other) if isinstance(other, str) else other
        self.append(other)
        return Dependency(self, other)
    
    def get_dependencies(self, relation : str) -> List['Dependency']:
        '''
        Extract dependents of this form for the specified dependency relation.
        '''
        deps = self.findall('*[@relation="{}"]'.format(relation))
        return [Dependency(self, dep, relation) for dep in deps]
    
    def __str__(self):
        return self.tag + '_' + self('id') + (('/' + self('pos')) if self('pos') else '')
    
    __repr__ = __str__
    
    def full_str(self, include_relation=True):
        '''
        Returns a string containing all features of the Form.
        The ID is attached to the text, and the relation is optionally suppressed.
        '''
        excluded = ['id', 'relation'] if not include_relation else ['id']
        return '{0}_{1} [{2}]'.format(self.tag, 
                                        self('id'), 
                                        ','.join([feature + '=' + self(feature) for feature in self.attrib.keys()
                                                 if feature not in excluded]))
    
    def __call__(self, feature : str) -> str:
        return self.get(feature)
    
    @staticmethod
    def to_form(word : stanfordnlp.pipeline.doc.Word) -> 'Form':
        '''
        Converts a stanfordnlp Word object to a Form
        '''
        form = Form(word.text, id = word.index)
        form.set('lemma', word.lemma)
        form.set('pos', word.pos)
        form.set('upos', word.upos)
        form.set('xpos', word.xpos)

        if word.feats != '_':
            for f in word.feats.split('|'):
                feature = f.split('=')
                form.set(feature[0], feature[1])

        return form
    
f = Form

In the simplest case, a `Form` is created from a string:

In [199]:
f('described')

described_0

Note that each `Form` has an ID (relative to its sentence, defaulting to zero).

We use `set` and `get` methods on forms to set its attributes.
`/` is a shortcut for setting the POS feature of the form.  
The form is callable, with a string feature, aliasing the `get` method.

In [200]:
desc_form = f('described')
desc_form.set('Tense', 'Past')
desc_form / 'VBN'
desc_form.full_str() # Form.full_str() pulls in all feature speficications

'described_0 [Tense=Past,pos=VBN]'

More typically a word is created from a stanfordnlp `Word`:

In [201]:
desc_form = f.to_form(doc.sentences[0].words[10])
print(desc_form)
print(desc_form.full_str())

described_11/VBN
described_11 [lemma=describe,pos=VBN,upos=VERB,xpos=VBN,Tense=Past,VerbForm=Part]


## Dependencies

A dependency is just a triple `(head, dep, relation)`, where the relation can be null, for an untyped dependency.

In [202]:
class Dependency:
    def __init__(self, head : Form, dep : Form, relation : str = None) -> None:
        self.head = head
        self.dep = dep
        self.relation = relation
        
    def __str__(self):
        return '{0}({1}, {2})'.format(self.relation if self.relation else '', self.head, self.dep)
    
    __repr__ = __str__
    
    def __or__(self, relation : str) -> Dependency:
        self.relation = relation
        self.dep.set('relation', relation)
        return self

The ovreloaded `>>` operator on `Form` creates a `Dependency` and the `|` operator on `Dependency` sets the relation.  Thus we can write:

In [203]:
adep = f('wrote') / 'VBN' >> f('Caesar') / 'NNP' | 'nsubj'
adep

nsubj(wrote_0/VBN, Caesar_0/NNP)

Importantly, these operator bear side-effects: the dependent form is made a child of the head, and the `relation` attribute is set in the dependent. 

In [204]:
adep.head[0], adep.dep.get('relation')

(Caesar_0/NNP, 'nsubj')

## Trees

   A `DependencyTree` is just a specialization of Python's `ElementTree`.

In [205]:
class DependencyTree(ElementTree):
    def __init__(self, root : Form) -> None:
        root.set('relation', 'root')
        
        ElementTree.__init__(self, root)
        
    def _get_deps(self, node : Form, deps : List[Dependency]) -> List[Dependency]:
        for child_node in node.getchildren():
            deps = self._get_deps(child_node, deps)
            deps.extend(node.get_dependencies(child_node('relation')))
        return deps
        
    def get_dependencies(self) -> List[Dependency]:
        '''
        Returns a list of all the dependency relations in the tree, generated by depth-first search.
        '''
        deps = self._get_deps(self.getroot(), [])
        deps.append(Dependency(None, self.getroot(), 'root'))
        return deps
    
    def _print_treelet(self, node : Form, indent : int, all_features : bool):
        edge = '└─ ' if indent > 0 else ''
        node_str = node.full_str(False) if all_features else str(node)
        print(' ' * indent + edge + node('relation') + ' | ' + node_str)
        
        for child_node in node.getchildren():
            self._print_treelet(child_node, indent + 4, all_features)
    
    def print_tree(self, all_features : bool = True):
        '''
        Prints a prety-printed (indented) representation of the dependency tree.
        If all_features is True, then each node is printed with its complete feature bundle.
        '''
        self._print_treelet(self.getroot(), indent = 0, all_features = all_features) 
        
    @staticmethod
    def to_tree(sentence : stanfordnlp.pipeline.doc.Sentence) -> 'DependencyTree':
        '''
        Factory method to create trees from stanfordnlp sentence parses.
        '''
        forms = {}
        for word in sentence.words:
            forms[word.index] = Form.to_form(word)

        for word in sentence.words:
            if word.dependency_relation == 'root':
                root = forms[word.index]
            else:
                gov = forms[str(word.governor)]
                dep = forms[word.index]
                gov >> dep | word.dependency_relation
            
        return Tree(root)

To see how this works, let's build up a tree from scratch -- presumably not a typical process for end users.

Note that in dependency linguistics the root node, marked by the `root` relation and having a null governor, is typically the *head of the predicate*.

In [230]:
john, loves, mary = f('John', 1) / 'NNP', f('loves', 2) / 'VRB', f('Mary', 3) / 'NNP'
loves >> john | 'nsubj'
loves >> mary | 'obj'
t = DependencyTree(loves)
t.print_tree(False)

root | loves_2/VRB
    └─ nsubj | John_1/NNP
    └─ obj | Mary_3/NNP


/home/jds/devworld/cltk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


In [231]:
t.get_dependencies()

/home/jds/devworld/cltk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  


[nsubj(loves_2/VRB, John_1/NNP),
 obj(loves_2/VRB, Mary_3/NNP),
 root(None, loves_2/VRB)]

Searching this tree is trivial, but demonstrates the use of XPath.  The following finds all nominal subject(s) and the governor(s) of subjects: 

In [232]:
t.findall('.//*[@relation="nsubj"]'), t.findall('.//*[@relation="nsubj"]/..')

([John_1/NNP], [loves_2/VRB])

In [233]:
t1 = DependencyTree.to_tree(doc.sentences[0])

In [235]:
t1.print_tree()

root | turned_31 [lemma=turn,pos=VBD,upos=VERB,xpos=VBD,Mood=Ind,Tense=Past,VerbForm=Fin]
    └─ obl | summer_3 [lemma=summer,pos=NN,upos=NOUN,xpos=NN,Number=Sing]
        └─ case | In_1 [lemma=in,pos=IN,upos=ADP,xpos=IN]
        └─ det | the_2 [lemma=the,pos=DT,upos=DET,xpos=DT,Definite=Def,PronType=Art]
        └─ nmod | year_7 [lemma=year,pos=NN,upos=NOUN,xpos=NN,Number=Sing]
            └─ case | of_4 [lemma=of,pos=IN,upos=ADP,xpos=IN]
            └─ det | the_5 [lemma=the,pos=DT,upos=DET,xpos=DT,Definite=Def,PronType=Art]
            └─ amod | Roman_6 [lemma=Roman,pos=JJ,upos=ADJ,xpos=JJ,Degree=Pos]
            └─ nummod | 699_8 [lemma=699,pos=CD,upos=NUM,xpos=CD,NumType=Card]
    └─ punct | ,_9 [lemma=,,pos=,,upos=PUNCT,xpos=,]
    └─ advcl | described_11 [lemma=describe,pos=VBN,upos=VERB,xpos=VBN,Tense=Past,VerbForm=Part]
        └─ advmod | now_10 [lemma=now,pos=RB,upos=ADV,xpos=RB]
        └─ obl | year_14 [lemma=year,pos=NN,upos=NOUN,xpos=NN,Number=Sing]
            └─ case |

/home/jds/devworld/cltk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


Let's find all obliques in this sentence:

In [240]:
t1.findall('.//*[@relation="obl"]')

[summer_3/NN, year_14/NN, birth_18/NN, Britain_35/NNP]

Or the children of "Gaul", and its parent, and all proper nouns:

In [244]:
t1.findall('.//Gaul/*'), t1.find('.//Gaul/..'), t1.findall('.//*[@pos="NNP"]')

([of_24/IN, Gaius_27/NNP],
 Proconsul_23/NNP,
 [Christ_20/NNP,
  Proconsul_23/NNP,
  Gaul_25/NNP,
  Gaius_27/NNP,
  Julius_28/NNP,
  Caesar_29/NNP,
  Britain_35/NNP])